In [1]:
import sys
sys.path.append('/home/trukhinmaksim/src')

In [2]:
import numpy as np
from time import time
from random import sample, seed as randomSeed
from collections import defaultdict
from numpy import mean

In [3]:
from src.utils.DatabaseConnect import DatabaseConnect

# single machine setup (mongo is running here localy)
# "ip a" for ip address
MY_DATABASE_LINK = 'mongodb://172.26.234.237:27020/' #'mongodb://192.168.100.57:27020/'
# multiple mechine setup (mongo is running on another machine)
#MY_DATABASE_LINK = 'mongodb://192.168.43.78:27020/'

DatabaseConnect.DB_LINK = MY_DATABASE_LINK

projectsCollection = DatabaseConnect.mini_database.projects()
usersCollection = DatabaseConnect.mini_database.users()
print(projectsCollection)

Collection(Database(MongoClient(host=['172.26.234.237:27020'], document_class=dict, tz_aware=False, connect=True), 'mini_database'), 'projects')


In [4]:
from src.utils.CacheAdapter import JSONAdapter, JSONMultiFileAdapter, EXP_END_OF_DATA
from src.utils.DatasetManager import ProjectsDatasetManager
from src.utils.validators import projectDataIsSufficient

In [5]:
def feedTextData(manager, batchSize = 1):
    # feeds text data by batches
    tempStorage = []

    i = 0
    while True:
        try:
            while len(tempStorage) >= batchSize:
                if batchSize > 1:
                    yield tempStorage[:batchSize]
                else:
                    yield tempStorage[:batchSize][0] # if I'm requesting only one item per time (training one by one) -> just yield it
                tempStorage = tempStorage[batchSize:]

            manager.fromDB()
            data = flatternData(manager.getTextOnly())
            tempStorage.extend(data)

            i += 1

        except EXP_END_OF_DATA:
            break

    yield tempStorage


In [6]:
def flatternData(data : dict[str, list]) -> np.array(dict):
    # takes in data in form of dict, where each key is a user id and each value is a list of that user's projects
    # returns just flat list of these projects 
    result = []

    for projectsArray in data.values():
        for project in projectsArray:
            result.append(project)

    return result

In [7]:
import gensim
from gensim.models.doc2vec import TaggedDocument

In [8]:
CACHE_FILE_NAME = "cache__02-04-2025__(good)_{0}.json"

In [9]:
# using adapter to load data from the cache files

# TODO: place implementation of the 'Corpus' class into a separate file
class Corpus:
    # base class for every data corpus, that will be used by model
    def __init__(self):
        pass
    def __iter__(self):
        pass
    def __getitem__(self, index : int):
        pass

class CacheCorpus(Corpus):
    def __init__(self, manager, cacheFileNameTemplate = CACHE_FILE_NAME, limit = float("inf")):
        self.cacheFileNameTemplate = cacheFileNameTemplate
        self.manager = manager # manager is needed not only for interaction with adapter, but also if I want to use unpreprocessed dataset and preprocess it on the way
        self.limit = limit

    #def __iter__(self):
    #    return self

    def __iter__(self):
        # will feed preprocessed projects data as TaggedDocument instances one by one
        cacheFileName = self.cacheFileNameTemplate
        tempStorage = [] # temporary storage for data, that was read from files

        i = 0
        while True:
            try:
                while len(tempStorage) >= 1:
                    doc = tempStorage[0]
                    yield TaggedDocument(words = doc["tokens"], tags = doc["tags"])
                    i += 1
                    if i >= self.limit:
                        raise EXP_END_OF_DATA

                    tempStorage = tempStorage[1:]

                #self.manager.cacheAdapter.collectionName = cacheFileName.format(i)
                data = flatternData(self.manager.fromCache())
                tempStorage.extend(data)

            except EXP_END_OF_DATA:
            # no data left
                break

        print("EOI")
        i = 0
        tempStorage.clear()
        self.manager.cacheAdapter.reset()

In [10]:
class EXP_FEEDER_IS_NONE(Exception):
    def __init__(self):
        super().__init__("'Model.corpus' object must be an iterable structure, inherited from 'Corpus' class!")

class EXP_MANAGER_IS_NONE(Exception):
    def __init__(self):
        super().__init__("'Model.manager' object must be a DatasetManager instance!")


class Model(gensim.models.doc2vec.Doc2Vec):
    @staticmethod
    def create(**kwargs):
        model = Model(
            dm_dbow_mode = "DM", 
            pretrain_w2v = False,
            **kwargs
        )
        model.corpus = CacheCorpus(manager)

        return model
    
    def __init__(self, dm_dbow_mode = "DM", pretrain_w2v = False, alpha_init = 0.05, alpha_final = 0.001, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.corpus = None # corpus is an iterator(iterable class object), that will be used in "train" method of Doc2Vec model for data extraction
        self.alphaInit = alpha_init
        self.alphaFinal = alpha_final
        self.dmDbowMode = dm_dbow_mode
        self.pretrainW2V = pretrain_w2v
    
    def train(self):
        # will build vocabulary and train the model on corpus (corpus will be fed by corpus)
        import logging
        logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
        
        if not isinstance(self.corpus, Corpus): raise EXP_FEEDER_IS_NONE

        start = time()
        self.build_vocab(self.corpus)
        print(f"Vocabulary built in {time() - start} s")

        if self.dmDbowMode != "DM+DBOW":
            start = time()
            super().train(
                self.corpus, 
                total_examples = self.corpus_count, 
                epochs = self.epochs,
                start_alpha = self.alphaInit,
                end_alpha = self.alphaFinal
            )
            print(f"Training is completed in {time() - start} s")
        else:
            # combine DM and DBOW
            pass

    def assess(self, sampleNum = 5, silent = False, resetSeed = False, format = "full"):
        # simple test of model performance
        # take multiple documents from the training corpus and tries to find simillar in the dataset
        # format = "full" | "means list"

        log = lambda s: print(s) if not silent else None
        #performanceGrageScale = {50 : "Random", 60 : "Poor", 70 : "Bad", 80 : "Medium", 92 : "Optimal", 97 : "Perfect"}
        totalDocuments = self.corpus_count
        if resetSeed: randomSeed(42)
        indexes = sample(range(totalDocuments), sampleNum)
        print(indexes)
        if format == "full":
            stats = {}

        i = 0
        docs = []
        avgPerformances = []

        for doc in self.corpus:
            if i >= totalDocuments: break
            if i in indexes:
                vector = self.infer_vector(doc.words)
                sims = defaultdict(lambda: 0, self.dv.most_similar([vector], topn = totalDocuments))

                log(f"Assessing document {i} ({doc.tags}). Similarities by tags:")
                if format == "full":
                    stats[i] = {
                        "similarities by tags" : {},
                        "average" : 0
                    }

                for tag in doc.tags:
                    if format == "full":
                        stats[i]["similarities by tags"][tag] = sims[tag]
                    log(f"  {tag} : {sims[tag]}")

                avgPerformances.append(mean([sims[tag] for tag in doc.tags]))
                log(f"\n  Average similarity value: {avgPerformances[-1]}\n")
                if format == "full":
                    stats[i]["average"] = avgPerformances[-1]
            i += 1

        if format == "full":
            stats["Average accuracy"] = mean(avgPerformances)
            log(f"Average accuracy: {stats['Average accuracy']}")

            return stats
        else:
            return mean(avgPerformances)

    def evaluate(self):
        # will train the model on upon-selected set of parameters and test it's performance
        # TODO: complete this method to evaluate the model
        #self.corpus = CacheCorpus(manager, limit = 100)
        result = self.assess(6, silent = True, resetSeed = True, format = "mean")

        return result


In [11]:
adapter = JSONMultiFileAdapter(CACHE_FILE_NAME)
#ProjectsDatasetManager.usersCollection = usersCollection
#ProjectsDatasetManager.projectsCollection = projectsCollection
manager = ProjectsDatasetManager(50, cacheAdapter = adapter)

In [12]:
corpus = CacheCorpus(manager, limit = 5)

In [13]:
i = 0
for doc in corpus:
    i += 1
print(i)

EOI
5


In [14]:
from skopt.space import Real, Integer
from src.utils.AutoTuner import AutoTuner, Param

In [15]:
# creating model

VECTOR_SIZE = 7
EPOCHS_NUMBER = 1
WORD_MIN_COUNT = 5
WINDOW_SIZE = 7
NEGATIVE_SAMPLES_AMOUNT = 6
SUBSAMPLING_THRESHOLD = 1e-5
ALPHA_INIT = 0.05
ALPHA_FINAL = 0.00001
DM_DBOW_MODE = "DM" # "DBOW" "DM+DBOW"

# finetunning is done by twicking model parameters
model = Model(
    vector_size =  VECTOR_SIZE, 
    window =       WINDOW_SIZE, 
    min_count =    WORD_MIN_COUNT, 
    epochs =       EPOCHS_NUMBER, 
    dm_dbow_mode = DM_DBOW_MODE,
    negative =     NEGATIVE_SAMPLES_AMOUNT,
    sample =       SUBSAMPLING_THRESHOLD,
    alpha_init =   ALPHA_INIT,
    alpha_final =  ALPHA_FINAL
)
model.corpus = CacheCorpus(manager, limit = 50)
#model.assess()
model.train()
print(model.corpus_count)
#model.build_vocab(documentsCorpus)
#model.train(documentsCorpus, total_examples = model.corpus_count, epochs = model.epochs)

2025-04-09 13:58:18,590 : INFO : collecting all words and their counts
2025-04-09 13:58:18,598 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2025-04-09 13:58:18,599 : WARNING : More unique tags (360) than documents (50).
2025-04-09 13:58:18,602 : INFO : collected 307 word types and 360 unique tags from a corpus of 50 examples and 448 words
2025-04-09 13:58:18,604 : INFO : Creating a fresh vocabulary
2025-04-09 13:58:18,605 : INFO : Model lifecycle event {'msg': 'effective_min_count=5 retains 6 unique words (1.95% of original 307, drops 301)', 'datetime': '2025-04-09T13:58:18.605242', 'gensim': '4.3.3', 'python': '3.11.11 (main, Jan 14 2025, 05:22:51) [GCC 12.2.0]', 'platform': 'Linux-6.13.8-200.fc41.x86_64-x86_64-with-glibc2.36', 'event': 'prepare_vocab'}
2025-04-09 13:58:18,606 : INFO : Model lifecycle event {'msg': 'effective_min_count=5 leaves 45 word corpus (10.04% of original 448, drops 403)', 'datetime': '2025-04-09T13:58:18.606356', 'gensi

EOI
Vocabulary built in 0.024121522903442383 s
EOI
Training is completed in 0.011691570281982422 s
50


In [16]:
# autotunning model parameters

def createModel(**kwargs):
    model = Model(
                vector_size = VECTOR_SIZE,
                dm_dbow_mode = "DM", 
                alpha_init = ALPHA_INIT,
                alpha_final = ALPHA_FINAL,
                **kwargs
            )
    manager.cacheAdapter.reset()
    manager.clearData()
    model.corpus = CacheCorpus(manager)

    return model

tuner = AutoTuner(createModel, [
    Param(_name = "window",    _type = Integer,  _range = (5, 10),      _initial = 7),
    Param(_name = "min_count", _type = Integer,  _range = (7, 12),      _initial = 7),
    Param(_name = "epochs",    _type = Integer,  _range = (25, 45),     _initial = 25),
    Param(_name = "negative",  _type = Integer,  _range = (5, 11),      _initial = 5),
    Param(_name = "sample",    _type = Real,     _range = (1e-6, 1e-5), _initial = 1e-5),
])

2025-04-09 13:58:18,636 : INFO : Model lifecycle event {'params': 'Model<dm/m,d7,n5,w7,mc7,s1e-05,t3>', 'datetime': '2025-04-09T13:58:18.636922', 'gensim': '4.3.3', 'python': '3.11.11 (main, Jan 14 2025, 05:22:51) [GCC 12.2.0]', 'platform': 'Linux-6.13.8-200.fc41.x86_64-x86_64-with-glibc2.36', 'event': 'created'}


In [23]:
#model.assess(3, True, True, format = "mean")
model.evaluate()

[40, 7, 1, 17, 15, 14]
EOI


0.09417400638312605

In [20]:
2/0
i = 1
words = []
tags = []
for proj in model.corpus:
    if i <= 0: break
    words = proj.words
    tags = proj.tags
    i -= 1

print(tags)
vector = model.infer_vector(['awesome', 'code', 'streamer', 'list', 'code', 'streamer', 'multiple', 'plataforms', 'like', 'twitch', 'youtube'])
vector

ZeroDivisionError: division by zero

In [ ]:
model.dv.most_similar([vector], topn=len(model.dv))